### Data prep

- Downloaded CSV of travis dataset 
- use first 100 commits of dataset 

- extract commits id & project name of those commits
- use values to get corresponding pom files (previous and current)
- use differ to extract information from pom files
- use this as data and the build result of the 100 travis commits as labels

In [4]:
#!pip3 install pandas
import pandas as pd 

# TODO just use if original travistorrent dataset is present instead of smaller version 
# TODO think of duplicates in commitID
# size down original file
#df = pd.read_csv('/Users/Rina/Desktop/StudiAssistent/Projekt_Bewerbung/data/travistorrent_5_3_2016.csv')
#df = df.drop_duplicates()[0:100]
#df.head()
#df.to_csv('travistorrent_100commits_withoutduplicates.csv', sep=',')


/var/folders/fq/98klxnfs5snds3nsbbmn85nw0000gn/T/ipykernel_609/4135246571.py:5: DtypeWarning: Columns (4,42,47,48,49,50,51,52,53,54) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/Users/Rina/Desktop/StudiAssistent/Projekt_Bewerbung/data/travistorrent_5_3_2016.csv')


In [ ]:
# read in file 
df = pd.read_csv('/Users/Rina/Desktop/StudiAssistent/Projekt_Bewerbung/dataprep/travistorrent_100commits_withoutduplicates.csv')
print(df.describe())

In [6]:
df.head()

,row,git_commit,gh_project_name,gh_is_pr,git_merged_with,gh_lang,git_branch,gh_first_commit_created_at,gh_team_size,git_commits,...,tr_tests_skipped,tr_failed_tests,tr_testduration,tr_purebuildduration,tr_tests_ran,tr_tests_failed,git_num_committers,tr_num_jobs,tr_prev_build,tr_ci_latency
0,1,7086c22bb72fa58e07968a292b3c06adce0491c2,47deg/appsly-android-rest,False,NaN,java,master,2013-08-17 00:41:08,3,c069748ad3fc29acf59422faeeac87281ea98a19,...,NaN,NaN,NaN,NaN,False,False,1,4,-1.0,-1
1,2,7086c22bb72fa58e07968a292b3c06adce0491c2,47deg/appsly-android-rest,False,NaN,java,master,2013-08-17 00:41:08,3,c069748ad3fc29acf59422faeeac87281ea98a19,...,NaN,NaN,NaN,NaN,False,False,1,4,-1.0,-1
2,3,7086c22bb72fa58e07968a292b3c06adce0491c2,47deg/appsly-android-rest,False,NaN,java,master,2013-08-17 00:41:08,3,c069748ad3fc29acf59422faeeac87281ea98a19,...,NaN,NaN,NaN,NaN,False,False,1,4,-1.0,-1
3,4,7086c22bb72fa58e07968a292b3c06adce0491c2,47deg/appsly-android-rest,False,NaN,java,master,2013-08-17 00:41:08,3,c069748ad3fc29acf59422faeeac87281ea98a19,...,NaN,NaN,NaN,NaN,False,False,1,4,-1.0,-1
4,5,9766eb141693fceb2340aa19059ac7081eb0b215,47deg/appsly-android-rest,False,NaN,java,master,2013-08-17 00:42:26,3,7086c22bb72fa58e07968a292b3c06adce0491c2,...,NaN,NaN,NaN,NaN,False,False,1,4,4277.0,-1


#### Get DataFrame with information on project and commit of which we have build results

In [7]:
commit_information = pd.DataFrame(df)
commit_information = commit_information[['gh_project_name', 'git_commit']]#.drop_duplicates()[0:100]
print(commit_information) # now we have a dataframe containing 
print('Size of dataframe (rows): ', commit_information.shape[0])

              gh_project_name                                git_commit
0   47deg/appsly-android-rest  7086c22bb72fa58e07968a292b3c06adce0491c2
1   47deg/appsly-android-rest  7086c22bb72fa58e07968a292b3c06adce0491c2
2   47deg/appsly-android-rest  7086c22bb72fa58e07968a292b3c06adce0491c2
3   47deg/appsly-android-rest  7086c22bb72fa58e07968a292b3c06adce0491c2
4   47deg/appsly-android-rest  9766eb141693fceb2340aa19059ac7081eb0b215
..                        ...                                       ...
95  47deg/appsly-android-rest  876a83a36beb5be55ef44451056674517b35f688
96  47deg/appsly-android-rest  876a83a36beb5be55ef44451056674517b35f688
97  47deg/appsly-android-rest  876a83a36beb5be55ef44451056674517b35f688
98  47deg/appsly-android-rest  11f1c1741701a578d73976c48508f439e5d86457
99  47deg/appsly-android-rest  11f1c1741701a578d73976c48508f439e5d86457

[100 rows x 2 columns]
Size of dataframe (rows):  100


#### Clone project, checkout commit id and download current pom and pom before the commit 

In [62]:
# Imports
#!pip3 install GitPython
import git 
import shutil
import os

In [63]:
# path to pom file of a project using maven 
prefix_project = '/Users/Rina/Desktop/StudiAssistent/Projekt_Bewerbung/dataprep/'
path_to_pom = prefix_project+'projects/pom.xml'
prefix_url = 'https://github.com/'

# method for getting Pom file from commit id 
import shutil
def getPomFile(project, commit_id, path_to_pom, path_to_save_to):
    project.git.checkout(commit_id) # checkout commit of data to receive pom.xml file
    shutil.copy(path_to_pom, path_to_save_to)
    #!cp path_to_pom path_to_save_to

In [66]:
for index, row in commit_information.iterrows():
    # do it for every commit 
    commit_id = row['git_commit']
    project_name = row['gh_project_name']

    project_url = prefix_url+project_name
    shutil.rmtree('projects')
    project = git.Repo.clone_from(project_url, '/Users/Rina/Desktop/StudiAssistent/Projekt_Bewerbung/dataprep/projects')

    # TODO adapt naming of subfolders (commit id's are present multiple times so i stayed with indices for now)
    path = prefix_project+'pom_files/'+str(index)#+'/'+project_name+'/'+commit_id
    os.makedirs(path)

    path_to_save_to_current = path+'/pom_current.xml'
    path_to_save_to_previous = path+'/pom_previous.xml'

    getPomFile(project, commit_id, path_to_pom, path_to_save_to_current)
    prev_commit_id = commit_id+'^1'
    project.git.checkout(prev_commit_id) # receive pom.xml file before commit was done
    getPomFile(project, prev_commit_id, path_to_pom, path_to_save_to_previous)

    # TODO run for all 100 commits to get all pom files into working directory for differ :) 
    if index == 1:
        break
    print(commit_id)


7086c22bb72fa58e07968a292b3c06adce0491c2
7086c22bb72fa58e07968a292b3c06adce0491c2
7086c22bb72fa58e07968a292b3c06adce0491c2
7086c22bb72fa58e07968a292b3c06adce0491c2
9766eb141693fceb2340aa19059ac7081eb0b215
9766eb141693fceb2340aa19059ac7081eb0b215
9766eb141693fceb2340aa19059ac7081eb0b215
9766eb141693fceb2340aa19059ac7081eb0b215


KeyboardInterrupt: 